# Lesson 7: SQL with Python - PostgreSQL Database Management

**Topic:** Using Python to interact with PostgreSQL databases

**Learning Objectives:**
- Connect to PostgreSQL databases using Python
- Create and manage database tables
- Execute SQL queries from Python
- Insert, update, and delete data
- Query data and convert to pandas DataFrames
- Understand database transactions and best practices

---

## Part 1: Setup and Installation

**Required Libraries:**
- `psycopg2` - PostgreSQL adapter for Python
- `pandas` - For data manipulation
- `sqlalchemy` - SQL toolkit and ORM (optional but recommended)

**Installation:**
```bash
pip install psycopg2-binary pandas sqlalchemy
```

In [ ]:
# Import required libraries
import psycopg2
from psycopg2 import sql, Error
import pandas as pd
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

## Part 2: Database Connection

**Connection Parameters:**
- `host` - Database server address (e.g., 'localhost')
- `database` - Database name
- `user` - Username
- `password` - Password
- `port` - Port number (default: 5432)

In [ ]:
# Database connection parameters
DB_CONFIG = {
    'host': 'localhost',
    'database': 'student_db',
    'user': 'postgres',
    'password': 'your_password',
    'port': 5432
}

# Function to create database connection
def get_connection():
    """Create and return a database connection"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("✅ Database connection successful!")
        return conn
    except Error as e:
        print(f"❌ Error connecting to database: {e}")
        return None

# Test connection
conn = get_connection()
if conn:
    conn.close()
    print("✅ Connection test passed!")

## Part 3: Creating a Database

**Note:** To create a database, you must connect to the default 'postgres' database first.

In [ ]:
def create_database(db_name):
    """Create a new PostgreSQL database"""
    try:
        # Connect to default postgres database
        conn = psycopg2.connect(
            host=DB_CONFIG['host'],
            database='postgres',
            user=DB_CONFIG['user'],
            password=DB_CONFIG['password'],
            port=DB_CONFIG['port']
        )
        conn.autocommit = True
        cursor = conn.cursor()
        
        # Check if database exists
        cursor.execute("SELECT 1 FROM pg_database WHERE datname = %s", (db_name,))
        exists = cursor.fetchone()
        
        if not exists:
            cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
            print(f"✅ Database '{db_name}' created successfully!")
        else:
            print(f"ℹ️  Database '{db_name}' already exists.")
        
        cursor.close()
        conn.close()
        
    except Error as e:
        print(f"❌ Error creating database: {e}")

# Create the database
create_database('student_db')

## Part 4: Creating Tables

**SQL Data Types:**
- `INTEGER` - Whole numbers
- `VARCHAR(n)` - Variable-length text (max n characters)
- `TEXT` - Unlimited text
- `DECIMAL(p,s)` - Decimal numbers
- `DATE` - Date values
- `TIMESTAMP` - Date and time
- `BOOLEAN` - True/False

In [ ]:
# SQL to create customers table
create_customers_table = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    city VARCHAR(50),
    state VARCHAR(2),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

# SQL to create orders table
create_orders_table = """
CREATE TABLE IF NOT EXISTS orders (
    order_id SERIAL PRIMARY KEY,
    customer_id INTEGER REFERENCES customers(customer_id),
    order_date DATE NOT NULL,
    total_amount DECIMAL(10, 2) NOT NULL,
    status VARCHAR(20) DEFAULT 'pending'
);
"""

# SQL to create products table
create_products_table = """
CREATE TABLE IF NOT EXISTS products (
    product_id SERIAL PRIMARY KEY,
    product_name VARCHAR(100) NOT NULL,
    category VARCHAR(50),
    price DECIMAL(10, 2) NOT NULL,
    stock_quantity INTEGER DEFAULT 0
);
"""

# Function to execute SQL
def execute_sql(sql_query, fetch=False):
    """Execute SQL query and optionally fetch results"""
    conn = get_connection()
    if not conn:
        return None
    
    try:
        cursor = conn.cursor()
        cursor.execute(sql_query)
        
        if fetch:
            results = cursor.fetchall()
            cursor.close()
            conn.close()
            return results
        else:
            conn.commit()
            cursor.close()
            conn.close()
            print("✅ SQL executed successfully!")
            
    except Error as e:
        print(f"❌ Error executing SQL: {e}")
        conn.rollback()
        conn.close()

# Create tables
execute_sql(create_customers_table)
execute_sql(create_orders_table)
execute_sql(create_products_table)

## Part 5: Inserting Data

**Methods:**
1. Single row insert
2. Multiple row insert
3. Insert from pandas DataFrame

In [ ]:
# Insert single customer
insert_customer = """
INSERT INTO customers (name, email, city, state)
VALUES (%s, %s, %s, %s)
RETURNING customer_id;
"""

def insert_data(query, data):
    """Insert data and return the inserted ID"""
    conn = get_connection()
    if not conn:
        return None
    
    try:
        cursor = conn.cursor()
        cursor.execute(query, data)
        inserted_id = cursor.fetchone()[0]
        conn.commit()
        cursor.close()
        conn.close()
        print(f"✅ Data inserted successfully! ID: {inserted_id}")
        return inserted_id
    except Error as e:
        print(f"❌ Error inserting data: {e}")
        conn.rollback()
        conn.close()
        return None

# Insert a customer
customer_data = ('John Doe', 'john.doe@email.com', 'New York', 'NY')
customer_id = insert_data(insert_customer, customer_data)

In [ ]:
# Insert multiple customers
def insert_many(query, data_list):
    """Insert multiple rows at once"""
    conn = get_connection()
    if not conn:
        return
    
    try:
        cursor = conn.cursor()
        cursor.executemany(query, data_list)
        conn.commit()
        print(f"✅ {cursor.rowcount} rows inserted successfully!")
        cursor.close()
        conn.close()
    except Error as e:
        print(f"❌ Error inserting data: {e}")
        conn.rollback()
        conn.close()

# Sample data
customers_data = [
    ('Jane Smith', 'jane.smith@email.com', 'Los Angeles', 'CA'),
    ('Bob Johnson', 'bob.j@email.com', 'Chicago', 'IL'),
    ('Alice Williams', 'alice.w@email.com', 'Houston', 'TX')
]

insert_customer_query = """
INSERT INTO customers (name, email, city, state)
VALUES (%s, %s, %s, %s);
"""

insert_many(insert_customer_query, customers_data)

## Part 6: Querying Data

**SQL SELECT Statements:**
- `SELECT * FROM table` - Get all columns
- `WHERE` - Filter rows
- `ORDER BY` - Sort results
- `LIMIT` - Limit number of rows
- `JOIN` - Combine tables

In [ ]:
# Query all customers
def query_to_dataframe(query):
    """Execute query and return results as pandas DataFrame"""
    conn = get_connection()
    if not conn:
        return None
    
    try:
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df
    except Error as e:
        print(f"❌ Error querying data: {e}")
        conn.close()
        return None

# Get all customers
customers_df = query_to_dataframe("SELECT * FROM customers;")
print("\nAll Customers:")
print(customers_df)

In [ ]:
# Query with WHERE clause
query_ca_customers = """
SELECT customer_id, name, email, city
FROM customers
WHERE state = 'CA'
ORDER BY name;
"""

ca_customers = query_to_dataframe(query_ca_customers)
print("\nCustomers in California:")
print(ca_customers)

## Part 7: Updating and Deleting Data

**UPDATE Statement:**
```sql
UPDATE table_name
SET column1 = value1, column2 = value2
WHERE condition;
```

**DELETE Statement:**
```sql
DELETE FROM table_name
WHERE condition;
```

In [ ]:
# Update customer email
update_query = """
UPDATE customers
SET email = %s
WHERE customer_id = %s;
"""

def update_data(query, data):
    """Execute UPDATE query"""
    conn = get_connection()
    if not conn:
        return
    
    try:
        cursor = conn.cursor()
        cursor.execute(query, data)
        conn.commit()
        print(f"✅ {cursor.rowcount} row(s) updated successfully!")
        cursor.close()
        conn.close()
    except Error as e:
        print(f"❌ Error updating data: {e}")
        conn.rollback()
        conn.close()

# Update email for customer_id 1
update_data(update_query, ('john.doe.new@email.com', 1))

In [ ]:
# Delete customer
delete_query = """
DELETE FROM customers
WHERE customer_id = %s;
"""

def delete_data(query, data):
    """Execute DELETE query"""
    conn = get_connection()
    if not conn:
        return
    
    try:
        cursor = conn.cursor()
        cursor.execute(query, data)
        conn.commit()
        print(f"✅ {cursor.rowcount} row(s) deleted successfully!")
        cursor.close()
        conn.close()
    except Error as e:
        print(f"❌ Error deleting data: {e}")
        conn.rollback()
        conn.close()

# Example: Delete customer with ID 999 (if exists)
# delete_data(delete_query, (999,))

## Part 8: SQL Joins

**Join Types:**
- `INNER JOIN` - Returns matching rows from both tables
- `LEFT JOIN` - Returns all rows from left table
- `RIGHT JOIN` - Returns all rows from right table
- `FULL OUTER JOIN` - Returns all rows from both tables

In [ ]:
# First, insert some sample orders
insert_orders = """
INSERT INTO orders (customer_id, order_date, total_amount, status)
VALUES 
    (1, '2024-01-15', 150.00, 'completed'),
    (1, '2024-02-20', 75.50, 'pending'),
    (2, '2024-01-10', 200.00, 'completed'),
    (3, '2024-03-05', 125.75, 'shipped');
"""

execute_sql(insert_orders)

# Query with INNER JOIN
join_query = """
SELECT 
    c.customer_id,
    c.name,
    c.city,
    o.order_id,
    o.order_date,
    o.total_amount,
    o.status
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
ORDER BY c.name, o.order_date;
"""

orders_df = query_to_dataframe(join_query)
print("\nCustomers with Orders:")
print(orders_df)

## Part 9: Aggregation and Grouping

**Aggregate Functions:**
- `COUNT()` - Count rows
- `SUM()` - Sum values
- `AVG()` - Average values
- `MIN()` / `MAX()` - Minimum/Maximum values
- `GROUP BY` - Group results

In [ ]:
# Customer order summary
summary_query = """
SELECT 
    c.customer_id,
    c.name,
    COUNT(o.order_id) as total_orders,
    SUM(o.total_amount) as total_spent,
    AVG(o.total_amount) as avg_order_value,
    MAX(o.order_date) as last_order_date
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.name
ORDER BY total_spent DESC;
"""

summary_df = query_to_dataframe(summary_query)
print("\nCustomer Order Summary:")
print(summary_df)

## Part 10: Pandas Integration

**Using SQLAlchemy for easier pandas integration**

In [ ]:
# Create SQLAlchemy engine
engine = create_engine(
    f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
    f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

# Read data into DataFrame
customers_df = pd.read_sql_table('customers', engine)
print("\nCustomers DataFrame:")
print(customers_df.head())

# Write DataFrame to database
new_products = pd.DataFrame({
    'product_name': ['Laptop', 'Mouse', 'Keyboard'],
    'category': ['Electronics', 'Electronics', 'Electronics'],
    'price': [999.99, 29.99, 79.99],
    'stock_quantity': [50, 200, 150]
})

new_products.to_sql('products', engine, if_exists='append', index=False)
print("\n✅ Products inserted from DataFrame!")

## Part 11: Best Practices

**Security:**
1. Always use parameterized queries to prevent SQL injection
2. Never concatenate user input directly into SQL strings
3. Store credentials securely (environment variables, config files)

**Performance:**
1. Use indexes on frequently queried columns
2. Limit result sets when possible
3. Use connection pooling for multiple queries

**Transactions:**
1. Use transactions for related operations
2. Always close connections
3. Handle errors with try/except blocks

In [ ]:
# Transaction example
def transfer_order(from_customer_id, to_customer_id, order_id):
    """Transfer an order from one customer to another (atomic operation)"""
    conn = get_connection()
    if not conn:
        return False
    
    try:
        cursor = conn.cursor()
        
        # Start transaction (implicit)
        cursor.execute(
            "UPDATE orders SET customer_id = %s WHERE order_id = %s AND customer_id = %s",
            (to_customer_id, order_id, from_customer_id)
        )
        
        if cursor.rowcount == 0:
            raise Exception("Order not found or doesn't belong to source customer")
        
        # Commit transaction
        conn.commit()
        print("✅ Order transferred successfully!")
        cursor.close()
        conn.close()
        return True
        
    except Exception as e:
        # Rollback on error
        conn.rollback()
        print(f"❌ Transaction failed: {e}")
        cursor.close()
        conn.close()
        return False

# Example usage (commented out)
# transfer_order(1, 2, 1)

## Part 12: Cleanup

**Dropping tables and databases (use with caution!)**

In [ ]:
# Drop tables (commented out for safety)
# execute_sql("DROP TABLE IF EXISTS orders CASCADE;")
# execute_sql("DROP TABLE IF EXISTS customers CASCADE;")
# execute_sql("DROP TABLE IF EXISTS products CASCADE;")

print("⚠️  Cleanup commands are commented out for safety.")
print("Uncomment to drop tables if needed.")

## Summary

**Key Concepts Covered:**
1. ✅ Connecting to PostgreSQL from Python
2. ✅ Creating databases and tables
3. ✅ Inserting data (single and bulk)
4. ✅ Querying data with SELECT
5. ✅ Updating and deleting records
6. ✅ SQL joins and aggregations
7. ✅ Pandas integration
8. ✅ Transactions and error handling
9. ✅ Best practices and security

**Next Steps:**
- Practice with more complex queries
- Learn about indexes and query optimization
- Explore stored procedures and functions
- Study database design and normalization